In [10]:
import torch
import torch.nn.functional as F
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn
from torch.utils.data import DataLoader  
from torchvision import datasets
import torchvision.transforms as transforms  # 

In [11]:
from Triplet_Dataset import TripletData

In [12]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
device

device(type='cuda')

In [13]:
trainset =  TripletData('training_triples.csv')
trainloader= torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=0 )

In [14]:
class Encoding_Network(nn.Module):
    
    def __init__(self):
        
        super(Encoding_Network, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(7, 32, kernel_size=2), nn.ReLU(), nn.Conv2d(32, 64, kernel_size=2), nn.BatchNorm2d(64), nn.ReLU(),
                                 nn.MaxPool2d(2,2))
        self.linear = nn.Linear(576, 256)
        
        
    def forward(self,x):
        x = self.conv(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.linear(x))
      
        return x
        

In [15]:
test_data = torch.randn(1,7,8,8)
test_model = Encoding_Network()
test_model(test_data)

tensor([[1.8417, 0.0000, 0.0000, 0.1411, 0.0000, 1.0802, 1.6015, 0.6298, 0.0000,
         0.1423, 0.0000, 0.0000, 0.6015, 0.0000, 0.1212, 0.6173, 0.0000, 1.2153,
         0.0000, 0.7775, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1852, 0.1623,
         1.1759, 1.0392, 0.0000, 0.0000, 0.0531, 0.0000, 0.0000, 0.0811, 0.8081,
         0.5548, 0.7960, 0.0000, 0.0000, 0.6627, 0.7294, 0.1574, 0.9837, 0.0000,
         0.4418, 0.0000, 0.1674, 0.0521, 1.1730, 0.5207, 0.0000, 0.0000, 0.5048,
         0.7715, 1.5921, 0.9661, 0.2274, 0.6439, 0.1013, 0.0000, 0.5392, 1.2827,
         0.1592, 0.1052, 0.0965, 0.4440, 0.0000, 0.8552, 0.0671, 0.0000, 0.2993,
         0.0000, 0.0000, 0.5037, 0.0000, 0.0000, 0.0000, 0.0000, 0.5714, 1.0979,
         0.6194, 0.0000, 0.9292, 0.0000, 0.9772, 1.0031, 0.1566, 0.9796, 0.0000,
         0.6631, 0.0000, 0.0000, 0.6284, 0.0000, 0.0000, 0.5403, 0.0000, 0.1231,
         0.2631, 0.3462, 0.0278, 0.9065, 0.5997, 0.0000, 0.0000, 0.0000, 1.1555,
         0.2930, 0.0000, 0.0

In [16]:
def train(model,criterion,optimizer,num_epochs,trainloader):
    
    for epoch in range(num_epochs):
        running_loss=0.0
        
        for i , (anchor, pos, neg) in enumerate(trainloader):
            anchor = anchor.to(device)
            pos = pos.to(device)
            neg = neg.to(device)
            
           
            #forward prop
            
            anchor_enc = model(anchor)
            pos_enc = model(pos)
            neg_enc = model(neg)
            

            loss = criterion(anchor_enc, pos_enc, neg_enc)

            #backprop
            optimizer.zero_grad()
            loss.backward()

            #take step
            optimizer.step()
            losss= loss.item()
            
            running_loss+=losss
            
            
           
            if i %500==499:
                print('[%d,%5d] loss: %.3f' % (epoch+1 , i+1, running_loss/500))
                running_loss=0.0
            
            

In [17]:
model = Encoding_Network().to(device)

In [19]:
batch_size = 16
learning_rate = 3e-4
num_epochs = 10
criterion3 = nn.TripletMarginLoss()
optimizer3=optim.Adam(model.parameters(),lr=learning_rate)
#optimizer3 = optim.SGD(my_model.parameters(),lr=learning_rate,momentum=.9)
num_epochs=15
train(model,criterion3,optimizer3,num_epochs,trainloader)

[1,  500] loss: 0.007
[1, 1000] loss: 0.019
[1, 1500] loss: 0.010
[1, 2000] loss: 0.004
[1, 2500] loss: 0.020
[1, 3000] loss: 0.006
[1, 3500] loss: 0.010
[1, 4000] loss: 0.020
[1, 4500] loss: 0.008
[1, 5000] loss: 0.009
[1, 5500] loss: 0.007
[1, 6000] loss: 0.004
[1, 6500] loss: 0.020
[1, 7000] loss: 0.006
[1, 7500] loss: 0.009
[1, 8000] loss: 0.008
[1, 8500] loss: 0.017
[1, 9000] loss: 0.006
[1, 9500] loss: 0.007
[1,10000] loss: 0.004
[1,10500] loss: 0.011
[1,11000] loss: 0.005
[1,11500] loss: 0.013
[1,12000] loss: 0.009
[1,12500] loss: 0.017
[1,13000] loss: 0.013
[1,13500] loss: 0.018
[1,14000] loss: 0.017
[1,14500] loss: 0.002
[1,15000] loss: 0.007
[1,15500] loss: 0.011
[1,16000] loss: 0.002
[1,16500] loss: 0.004
[1,17000] loss: 0.007
[1,17500] loss: 0.006
[1,18000] loss: 0.006
[1,18500] loss: 0.003
[1,19000] loss: 0.007
[1,19500] loss: 0.008
[1,20000] loss: 0.007
[1,20500] loss: 0.010
[1,21000] loss: 0.006
[1,21500] loss: 0.010
[1,22000] loss: 0.004
[1,22500] loss: 0.008
[1,23000] 

KeyboardInterrupt: 

In [20]:
torch.save(model.state_dict(), './previous_models/Vec2.pth')

In [ ]:
'''
Vec1.pth -> .4
class Encoding_Network(nn.Module):
    
    def __init__(self):
        
        super(Encoding_Network, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(7, 32, kernel_size=2), nn.ReLU(), nn.Conv2d(32, 64, kernel_size=2), nn.ReLU(),
                                 nn.MaxPool2d(2,2))
        self.linear = nn.Linear(576, 256)
        
    def forward(self,x):
        x = self.conv(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.linear(x))
        return x
        
vec2 -> .005        
class Encoding_Network(nn.Module):
    
    def __init__(self):
        
        super(Encoding_Network, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(7, 32, kernel_size=2), nn.ReLU(), nn.Conv2d(32, 64, kernel_size=2), nn.BatchNorm2d(64), nn.ReLU(),
                                 nn.MaxPool2d(2,2))
        self.linear = nn.Linear(576, 256)
        
        
    def forward(self,x):
        x = self.conv(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.linear(x))
      
        return x
        

'''